In [1]:
import uproot as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from global_settings import *

In [2]:
def logit_transform(score):
    return np.log(score/(1-score))

def filter_df(df):
    df = df[df['signal_score'] >= 0.5]
    return df

def GetStatsUncert(df_a, weight_array,my_custom_bins,plot_var):
    (counts_df, bins_df) = np.histogram(df_a[plot_var], bins = my_custom_bins, weights=np.square(weight_array))
    return counts_df

In [3]:
run="run1"
dm_type = 'fermion'
ratio = "2.0"



target_pot=0

scalar_factors = np.array([0.04444236, 0.03876295, 0.03149615, 0.03032621, 0.02560909, 0.02296708,
 0.0244469, 0.02523209, 0.03118899, 0.03338144, 0.02626005, 0.01194017,
 0.0040589 ])

    
if(run=="run1"):
    target_pot = 2.38e20
    
else:
    target_pot = 5.18e20 
    
print("Processing Run: ", run)
print("Target POT: ", target_pot)

Processing Run:  run1
Target POT:  2.38e+20


In [4]:
base_dir = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/{}_signal/".format(run)

In [5]:
def MakeSignalHist(run,dm_type,ratio,mass,dmode):
    my_custom_bins = final_custom_bins
    df_signal = pd.read_csv(base_dir + "{}_dt_ratio_{}_{}_{}_CNN_scores_8441_steps.csv".format(run,ratio,mass,dmode))
    df_signal = df_signal.drop_duplicates()
    signal_fraction = len(df_signal)/len(df_signal[df_signal['signal_score']>= 0.])
    signal_scaling = (meson_scalings[dmode]*signal_fraction*target_pot)/(df_signal['total_pot'][0])
    
    print("For mass, decay mode "+ mass + " " + dmode)
    print("1e21 Number of events as a reference: ", 1e21/df_signal['mctruth_w'][0])
    
    if(ratio==str(0.6)):
        signal_scaling*=correction_dic_a[dmode][mass]
    else:
        signal_scaling*=correction_dic_b[dmode][mass]
        
    if(dm_type == "scalar" and ratio == str(0.6)):
        signal_scaling*=scalar_dic_a[dmode][mass]
    elif(dm_type == "scalar" and ratio == str(2.0)):
        signal_scaling*=scalar_dic_b[dmode][mass]

    df_signal = filter_df(df_signal)
    df_signal['signal_score'] = logit_transform(df_signal['signal_score'])
    (counts_signal_temp, bins_signal) = np.histogram(df_signal['signal_score'], bins = my_custom_bins )
    stats_uncert_temp = GetStatsUncert(df_signal,np.ones(len(df_signal))*signal_scaling,my_custom_bins,'signal_score')
    return (counts_signal_temp,stats_uncert_temp,bins_signal,signal_scaling)
    

In [6]:
def MakeSignalFile(run,dm_type,ratio):
    my_custom_bins = final_custom_bins
    signal_file = u.recreate(base_dir + "{}_ratio_{}_signal_hist_{}_CNN.root".format(dm_type,ratio,run))
    
    # Retrieve array of masses 
    masses = mass_dic[ratio]
    
    # Decay types
    decays = ["pi0", "eta"]
    
    
    # Retrieve dictionary with xsec uncerts
    if(run=="run1" and ratio == "0.6"):
        xsec_uncertainties = run1_xsec_uncert_a
    elif(run=="run3" and ratio == "0.6"):
        xsec_uncertainties = run3_xsec_uncert_a
    elif(run=="run1" and ratio == "2.0"):
        xsec_uncertainties = run1_xsec_uncert_b
    elif(run=="run3" and ratio == "2.0"):
        xsec_uncertainties = run3_xsec_uncert_b
    
    # Create a histogram for each mass point 
    for mass in masses:        
        counts_signal = np.zeros(len(my_custom_bins) - 1) 
        stats_uncert = np.zeros(len(my_custom_bins) - 1)
        xsec_uncert  = 0
        #xsec_uncert = np.sqrt()
        
        
        if(ratio == "0.6"):
            if(float(mass) > 0.01 and float(mass)< 0.1):
                for dmode in decays:
                    # Adding xsec error in quadrature
                    xsec_uncert += np.square(xsec_uncertainties[dmode][mass])
                    (counts_signal_temp, stats_uncert_temp, bins_signal,signal_scaling) = MakeSignalHist(run,dm_type,ratio,mass,dmode)
                    counts_signal+=counts_signal_temp*signal_scaling
                    stats_uncert+=stats_uncert_temp


            elif(float(mass) == 0.01):
                dmode=decays[0]
                # Adding xsec error in quadrature
                xsec_uncert += np.square(xsec_uncertainties[dmode][mass])
                (counts_signal_temp, stats_uncert_temp, bins_signal,signal_scaling) = MakeSignalHist(run,dm_type,ratio,mass,dmode)
                counts_signal+=counts_signal_temp*signal_scaling
                stats_uncert+=stats_uncert_temp

            elif(float(mass) >= 0.1):
                dmode=decays[1]
                # Adding xsec error in quadrature
                xsec_uncert += np.square(xsec_uncertainties[dmode][mass])
                (counts_signal_temp, stats_uncert_temp, bins_signal,signal_scaling) = MakeSignalHist(run,dm_type,ratio,mass,dmode)
                counts_signal+=counts_signal_temp*signal_scaling
                stats_uncert+=stats_uncert_temp
        else:
            if(float(mass) <= 0.03):
                for dmode in decays:
                    # Adding xsec error in quadrature
                    xsec_uncert += np.square(xsec_uncertainties[dmode][mass])
                    (counts_signal_temp, stats_uncert_temp, bins_signal,signal_scaling) = MakeSignalHist(run,dm_type,ratio,mass,dmode)
                    counts_signal+=counts_signal_temp*signal_scaling
                    stats_uncert+=stats_uncert_temp
                
            else:
                dmode=decays[1]
                # Adding xsec error in quadrature
                xsec_uncert += np.square(xsec_uncertainties[dmode][mass])
                (counts_signal_temp, stats_uncert_temp, bins_signal,signal_scaling) = MakeSignalHist(run,dm_type,ratio,mass,dmode)
                counts_signal+=counts_signal_temp*signal_scaling
                stats_uncert+=stats_uncert_temp
        
        
        # Total uncertainty in %
        total_uncert = np.sqrt((stats_uncert/(counts_signal))*100 + np.ones(len(bins_signal)-1)*xsec_uncert + np.ones(len(bins_signal)-1)*(detvar_uncert*100)**2)
        stats_uncert = np.sqrt(stats_uncert)
        xsec_uncert = np.sqrt(xsec_uncert)
        
        print("Mass: " + mass)
        print("Total events for target POT: ", np.sum(counts_signal))
        print("Statistical uncertainty: ",stats_uncert)
        print("CV bins: ", counts_signal)
        print("Statistical fractional error: ", (stats_uncert/(counts_signal))*100)
        print("xsec error: ", xsec_uncert)
        print("Total error: ", total_uncert)
        print("\n")
        signal_file["signal_"+mass] = (counts_signal, bins_signal)
        signal_file["signal_mc_error_"+mass] = (stats_uncert, bins_signal)
        signal_file["signal_xsec_error_"+mass] = (np.ones(len(bins_signal)-1)*xsec_uncert,bins_signal)
        signal_file["signal_det_uncert_"+mass] = (np.ones(len(bins_signal)-1)*detvar_uncert,bins_signal)
        signal_file["signal_total_error_"+mass] = (total_uncert, bins_signal)

In [7]:
MakeSignalFile(run,dm_type,ratio)

For mass, decay mode 0.010 pi0
1e21 Number of events as a reference:  2763.9656273234586
For mass, decay mode 0.010 eta
1e21 Number of events as a reference:  664.4827334161722
Mass: 0.010
Total events for target POT:  25683.10945951785
Statistical uncertainty:  [167.75493895 185.31864217 197.30410383 177.9863639  124.57188505]
CV bins:  [4816.29420032 5899.60708213 6734.01638478 5492.20909344 2740.98269884]
Statistical fractional error:  [3.48307084 3.14120313 2.92996174 3.24070626 4.54478918]
xsec error:  1.9720796506246085
Total error:  [30.20250709 30.16641504 30.09954442 30.07807543 29.900536  ]


For mass, decay mode 0.020 pi0
1e21 Number of events as a reference:  45.91073117430468
For mass, decay mode 0.020 eta
1e21 Number of events as a reference:  86.12374259335813
Mass: 0.020
Total events for target POT:  640.2975779849525
Statistical uncertainty:  [3.09568509 3.5635098  3.79874689 3.603713   2.64316582]
CV bins:  [109.4042223  144.22666491 164.01316849 145.11641218  77.5371

For mass, decay mode 0.115 eta
1e21 Number of events as a reference:  6.717044500419816e-05
Mass: 0.115
Total events for target POT:  0.000183255272307238
Statistical uncertainty:  [1.40890554e-06 1.61080661e-06 1.69967805e-06 1.47767259e-06
 1.12176540e-06]
CV bins:  [3.33407903e-05 4.35811759e-05 4.85227573e-05 3.66748693e-05
 2.11356796e-05]
Statistical fractional error:  [4.22577127 3.69610635 3.50284722 4.02911482 5.30744892]
xsec error:  12.69399184519348
Total error:  [22.02583562 22.02583562 22.02583562 22.02583562 22.02583562]


For mass, decay mode 0.120 eta
1e21 Number of events as a reference:  1.6647494302395075e-05
Mass: 0.120
Total events for target POT:  4.055862782036435e-05
Statistical uncertainty:  [3.15571252e-07 3.66686091e-07 3.83101780e-07 3.30861211e-07
 2.52176141e-07]
CV bins:  [7.29235935e-06 9.84605073e-06 1.07473536e-05 8.01613284e-06
 4.65673134e-06]
Statistical fractional error:  [4.32742322 3.72419461 3.56461503 4.12744172 5.41530361]
xsec error:  12.483